# Sparkify with Casandra

## Part I: creating ETL process

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

### Creating filepaths to process original csv files

In [2]:
# Checking current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

### Processing files to create data file csv used for Apache Casandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# Extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# Checking the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Part II: Creating and connecting to database

In [5]:
#Creating cluster
from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [6]:
#Creating keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS casproject 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [7]:
#Setting keyspace
try:
    session.set_keyspace('casproject')
except Exception as e:
    print(e)

## Part III: Creating tables for three queries

### Creating song_length table

In [8]:
#Table is modelled for a query for artist, song title and song length with a specified session ID and itemInSession

query = "CREATE TABLE IF NOT EXISTS song_length "
query = query + "(sessionID int, \
                  itemInSession int, \
                  artist varchar, \
                  song varchar, \
                  length float, \
                  PRIMARY KEY(sessionID, \
                  itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_length (sessionID, \
                                          itemInSession, \
                                          artist, \
                                          song, \
                                          length)"
        query = query + " VALUES (%s, \
                                  %s, \
                                  %s, \
                                  %s, \
                                  %s)"
        
        session.execute(query, (\
                                int(line[8]), \
                                int(line[3]), \
                                line[0], \
                                line[9], \
                                float(line[5])))

### Testing table with query its designed for

In [9]:
#The query retrieves name of artist, song and song length for a session with ID 338 and item in session being 4.

query = "SELECT artist, song, length FROM song_length WHERE sessionID=338 AND itemInSession=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Creating artist_song_user table

In [10]:
#Table is created for query for artist, song and user information in form of first and last name given a \
#specified userID and session 

query = "CREATE TABLE IF NOT EXISTS artist_song_user "
query = query + "(userID int, \
                  sessionID int, \
                  itemInSession int, \
                  artist varchar, \
                  song varchar, \
                  first_name varchar, \
                  last_name varchar, \
                  PRIMARY KEY((userID, sessionID), \
                  itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user (userID, \
                                               sessionID, \
                                               itemInSession, \
                                               artist, \
                                               song, \
                                               first_name, \
                                               last_name)"
        query = query + " VALUES (%s, \
                                  %s, \
                                  %s, \
                                  %s, \
                                  %s, \
                                  %s, \
                                  %s)"
        
        session.execute(query, (int(line[10]), \
                                int(line[8]), \
                                int(line[3]), \
                                line[0], \
                                line[9], \
                                line[1], \
                                line [4]))

### Testing table with its query

In [11]:
#The query retrieves names of artists and songs and the first and last names of user from artist_song_user table for\
# a user with userID of 10 while sessionID is 182

query = "SELECT artist, song, first_name, last_name FROM artist_song_user WHERE userID=10 AND sessionID=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Creating users_all_hands table

In [12]:
#Third table is created for a query for user information (user ID, first and last name) for a selected song
query = "CREATE TABLE IF NOT EXISTS users_all_hands "
query = query + "(song varchar, \
                  userID int, \
                  first_name varchar, \
                  last_name varchar, \
                  PRIMARY KEY(song, userID))"
try:
    session.execute(query)
except Exception as e:
    print(e)      

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_all_hands (song, \
                                              userID, \
                                              first_name, \
                                              last_name)"                                
        query = query + " VALUES (%s, \
                                  %s, \
                                  %s, \
                                  %s)"
        
        session.execute(query, (line[9], \
                                int(line[10]), \
                                line[1], \
                                line[4]))    

### Testing table with query

In [13]:
#The query retrieves the first and last names of those users, who listened to song All Hands Against His Own

query = "SELECT first_name, last_name FROM users_all_hands WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


## Part IV: Dropping tables and closing session and connection 

In [14]:
# Dropping tables
query = "drop table song_length"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table artist_song_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table users_all_hands"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [15]:
#Closing session and cluster connection
session.shutdown()
cluster.shutdown()